# Development 


## 2D to 3D  

### Primary Focus: Tomographic Reconstruction   
### Application: Tomographic Reconstruction of STEM tilt series

#### Challenge: Evaluation with missing ground truth    
#### Required Labels: None

TL;DR 🧬✨ We use deep learning for tomographic reconstruction of 2D STEM projections, following [1,2]. This approach enables 3D volume reconstruction, revealing detailed cellular structures and relationships not visible in 2D.

![Teaser](./images/Teaser.gif)





# Setup and Imports

In [1]:
# auto reload imports
%load_ext autoreload
%autoreload 2

# imports from the template 
from deepEM.Utils import create_text_widget, print_info, find_file
from deepEM.Logger import Logger
from deepEM.ModelTuner import ModelTuner

# costum implementation
from src.ModelTrainer import ModelTrainer


# import all required libraries
from pathlib import Path 



# 1. Data

## 1.1 Data Acquisition  

In the case of tomographic reconstruction we do not have access to ground truth information (the 3D structure of the underlying sample). Still, it is important to verify the applicability of the deep learning method. Hence, the use of synthetic data to prove the correctness of the approach and estimate errors. 
Luckily, we do not need to generate data from scratch, but we can make use of existing synthetic data [1].

*[1] Kniesel, Hannah, et al. "Clean implicit 3d structure from noisy 2d stem images." Proceedings of the IEEE/CVF Conference on Computer Vision and Pattern Recognition. 2022.*

## 1.2. Data Anntation

The applied deep learning method for tomographic reconstruction is a self-supervised appraoch, which means that we do not need annotated data during training of the neural network. 
Similarly, as we are working with synthetic data, no annotated data is needed for the evaluation.

## 1.3. Data Preprocessing

If you wish to reconstruct your own tomogram using the provided notebook, these are the preprocessing steps you need to do:

1. **Data Alignment**: As there are usually very small deviations of the image alignment between different acquistion angles, the data should be aligned before reconstruction. While [ImageJ](https://imagej.net/ij/) provides data alignment using SIFT, we usually recommend to use more sophisticated approaches like the tracking of gold particles for alignment (for example by using [iMOD](https://bio3d.colorado.edu/imod/)), to get the best results.

2. **Tilt Axis Correction**: In some cases, the tilt axis can be slightly tilted off the main central vertical axis. This needs to be corrected before applying the reconstruction algorithm.
Additionally, we require the tilt axis to be vertical. When a horizontal tilt axis is provided, software like [ImageJ](https://imagej.net/ij/) can be used to rotate the images.



## 1.4. Data Structuring

We require the data to be organized in a single folder containing a series of `.tif` files, which are sorted by their names. 

The folder further needs to contain a `.rawtlt` file with the tilt angles of the EM. When the data is currently in `.mrc` file format, this can be achieved by using the `mrc2tif` command of the [iMOD](https://bio3d.colorado.edu/imod/) software or by using the [ImageJ](https://imagej.net/ij/) software. 

Lastly, we require a `metadata.json` with following content:

```json 
{
    "slice_thickness_nm": 550,
    "pixelsize_nmperpixel": 1.0,
    "original_px_resolution": 1000
}
```

 - `slice_theickness_nm` is the approximated slice thickness of your sample in [nm]. 
 - `pixelsize_nmperpixel` is the pixelsize of your dataset in [nm/px]. 
 - `original_px_resolution` is the image resolution of a single `.tif` in your tilt series in [px]. 


You can generate such file, within any text editor of your choice. Add the above lines of content and adapt the parameters based on your data. Save the file as `metadata.json`. 

### Synthetic Data
Due to the missing ground truth information on real data, model development will be done on synthetic data. The synthetic data consists of a noisy tilt series, which is used for training and a clean tilt series which is used for evaluation. Additionally, we use the phantom volume (hence the ground truth underlying sample of the synthetic data) for evaluation purposes.

An example with a tilt series of five EM images and the corresponding `.rawtlt` and `metadata.json` is shown below: 

```
data/
├── noisy-projections
    ├── image_001.tif
    ├── image_002.tif
    ├── image_003.tif
    ├── image_004.tif
    ├── image_005.tif
    ├── metadata.json
    └── tilts.rawtlt
├── clean-projections
    ├── image_001.tif
    ├── image_002.tif
    ├── image_003.tif
    ├── image_004.tif
    ├── image_005.tif
    ├── metadata.json
    └── tilts.rawtlt
└── phantom-volume
    └── volume.raw

```
For details please check the provided data within this use case.


If you are using different data than the one provided (`data/`), we require you to set the path to this folder. To do so, adapt the path in the text form below.


In [2]:
data_widget = create_text_widget("Data Path:","./data/synthetic","Enter the path to your data folder.")
display(*data_widget)

Text(value='./data/synthetic', description='Data Path:', style=TextStyle(description_width='initial'))

HTML(value='<b>Hint:</b> Enter the path to your data folder.')

In [3]:
data_path = data_widget[0].value
print(f"[INFO]::Data path was set to: {data_path}")

[INFO]::Data path was set to: ./data/synthetic


# 2. Model Training

## 2.1. Setup Logging

By executing the cell below, we setup the logging directory for model training and evaluation. 
The logger creates a folder at `./logs/<datafoldername>-<currentdatetime>/`. 
Within this folder there will be logging of: 

- the used hyperparameters, (`<log-path>/TrainingRun/hyperparameters.json`)
- the best performing model checkpoint based on the validation loss (`<log-path>/TrainingRun/checkpoints/best_model.pth`)
- the last model checkpoint (`<log-path>/TrainingRun/checkpoints/latest_model.pth`)
- visualizations of training and validation curves (`<log-path>/TrainingRun/plots/training_curves.png`)
- qualitative visualization of sampled validation images (`<log-path>/TrainingRun/samples/`)
- results on test metrics (`<log-path>/TrainingRun/test_results.txt`)
- qualitative visualization of sampled test images (`<log-path>/TrainingRun/samples/`)

Additional text logs are being saved to `<log-path>/TrainingRun/log.txt` and `<log-path>/log.txt` 

During validation we visualize a synthetic EM image, generated from the currently learned reconstruction and the corresponding real EM image next to it.

In [4]:
logger = Logger(data_path)

Logger initialized. Logs will be saved to: logs/synthetic_2025-03-25_17-49-28


## 2.2. Hyperparameter Tuning


Hyperparameters in deep learning are configurable settings that define how a model is trained. Unlike model parameters, they are set before training and not learned from the data.

When training a model, we highly recommend to do a hyperparameter tuning first. By tuning the hyperparameters the model is usually trained on a subset of the data with a smaller number of epochs, and then evaluated based on its performance on the validation set. Then, hyperparameters, which lead to the best performance are chosen for full training of the model. 
Similar to the training run, all sweep runs will be logged. You can find the according logs at `<log-path>/Sweep-<idx>`.

Our workflow equips you, as EM experts, with an automatic hyperparameter search based on a grid search. The DL experts have chosen some basic setting for performing the hyperparameter seach and defining the search space. The DL experts also describe the individual hyperparameters. This allows you to further adapt the search space to your specific needs. 

In order to do so, you can adapt the form below. Each sweep parameter should be separated by `,`. Floating point values should be written like `0.1`. 

In [5]:
# hyperparameter search
model_trainer = ModelTrainer(data_path, logger)

hyperparameter_tuner = ModelTuner(model_trainer, logger)
form = hyperparameter_tuner.create_hyperparameter_widgets()
display(form)


If you wish to run a hyperparameter sweep based on the parameters above, please execute the cell below.

In [6]:
best_config = None
hyperparameter_tuner.update_config(form)
print("Sweep config:")
for k in hyperparameter_tuner.config['hyperparameter'].keys():
    print(f"\t{k}: {hyperparameter_tuner.config['hyperparameter'][k]['values']} (default: {hyperparameter_tuner.config['hyperparameter'][k]['default']})")
best_config = hyperparameter_tuner.tune()

2025-03-25 17:49:28,208 - INFO - Start hyperparameter sweep...
2025-03-25 17:49:28,209 - INFO - Start Sweep 1 of 6...
2025-03-25 17:49:28,209 - INFO - Current hyperparams {'learning_rate': 0.0005, 'batch_size': 4, 'resize': 100}
2025-03-25 17:49:28,210 - INFO - Hyperparameters saved to logs/synthetic_2025-03-25_17-49-28/Sweep_0/hyperparameters.json


Sweep config:
	learning_rate: [0.0005, 5e-05, 5e-06] (default: 5e-05)
	batch_size: [4, 8] (default: 4)
	resize: [100] (default: 100)
Logger initialized. Logs will be saved to: logs/synthetic_2025-03-25_17-49-28/Sweep_0


2025-03-25 17:49:28,454 - INFO - Model was setup.


INFO::Found 94 images within tilt series at ./data/synthetic/noisy-projections/*.tif.
INFO::Resized image resolution from (1000, 1000) to (100, 100). Note that strong downscaling can lead to loss of information.
INFO::Applied minmax normalization to tilt series. 
 Current max = 1.0000 | Current min = 0.0000
INFO:: tilt angles were loaded from ./data/synthetic/noisy-projections/tilt.rawtlt
INFO::Found 94 images within tilt series at ./data/synthetic/clean-projections/*.tif.
INFO::Resized image resolution from (1000, 1000) to (100, 100). Note that strong downscaling can lead to loss of information.
INFO::Applied minmax normalization to tilt series. 
 Current max = 1.0000 | Current min = 0.0000
INFO:: tilt angles were loaded from ./data/synthetic/clean-projections/tilt.rawtlt
INFO::Use 20% for validation, resulting in 18 projection images.
INFO::Found 94 images within tilt series at ./data/synthetic/clean-projections/*.tif.
INFO::Resized image resolution from (1000, 1000) to (100, 100). N

2025-03-25 17:49:31,962 - INFO - Start Training | Epoch: 5 | Dataset size: 940000 | Parameters: {'epochs': 25, 'early_stopping_patience': 5, 'validation_interval': 5, 'scheduler_step_by': 'epoch', 'images_to_visualize': 4, 'pos_enc': 5, 'accum_gradients': 4, 'batch_size': 4, 'beam_samples': 64, 'learning_rate': 0.0005, 'resize': 100} 
/media/hansel/SSD/Code/PaulFestschrift/Deep-EM Playground/Tomogram/src/STEM.py:80: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:78.)
  offset = torch.cuda.FloatTensor(batch_size, beam_samples*2).uniform_(float(-bin_size/2), float(bin_size/2))
2025-03-25 18:07:55,229 - INFO - Epoch 0 - Training loss: 0.0059
2025-03-25 18:08:19,252 - INFO - Saved visualizations to logs/synthetic_2025-03-25_17-49-28/Sweep_0/samples/validation_epoch-0_*
2025-03-25 18:10:01,511 - INFO 

...

2025-03-25 22:09:22,267 - INFO - Start Training | Epoch: 5 | Dataset size: 940000 | Parameters: {'epochs': 25, 'early_stopping_patience': 5, 'validation_interval': 5, 'scheduler_step_by': 'epoch', 'images_to_visualize': 4, 'pos_enc': 5, 'accum_gradients': 4, 'batch_size': 8, 'beam_samples': 64, 'learning_rate': 5e-05, 'resize': 100} 
2025-03-25 22:19:00,958 - INFO - Epoch 0 - Training loss: 0.0071
2025-03-25 22:19:13,238 - INFO - Saved visualizations to logs/synthetic_2025-03-25_17-49-28/Sweep_3/samples/validation_epoch-0_*
2025-03-25 22:20:09,281 - INFO - Epoch 0 - Validation loss: 0.0079, MSE: 0.0079
2025-03-25 22:20:09,292 - INFO - Current model checkpoint saved to logs/synthetic_2025-03-25_17-49-28/Sweep_3/checkpoints/latest_model.pth
2025-03-25 22:20:09,303 - INFO - Best model checkpoint saved to logs/synthetic_2025-03-25_17-49-28/Sweep_3/checkpoints/best_model.pth (Validation Loss: 0.0079)
2025-03-25 22:20:09,304 - INFO - Avg time single epoch: 0h10m47s | Remaining time training:

Logger initialized. Logs will be saved to: logs/synthetic_2025-03-25_17-49-28/Sweep_4
INFO::Found 94 images within tilt series at ./data/synthetic/noisy-projections/*.tif.
INFO::Resized image resolution from (1000, 1000) to (100, 100). Note that strong downscaling can lead to loss of information.
INFO::Applied minmax normalization to tilt series. 
 Current max = 1.0000 | Current min = 0.0000
INFO:: tilt angles were loaded from ./data/synthetic/noisy-projections/tilt.rawtlt
INFO::Found 94 images within tilt series at ./data/synthetic/clean-projections/*.tif.
INFO::Resized image resolution from (1000, 1000) to (100, 100). Note that strong downscaling can lead to loss of information.
INFO::Applied minmax normalization to tilt series. 
 Current max = 1.0000 | Current min = 0.0000
INFO:: tilt angles were loaded from ./data/synthetic/clean-projections/tilt.rawtlt
INFO::Use 20% for validation, resulting in 18 projection images.
INFO::Found 94 images within tilt series at ./data/synthetic/clea

2025-03-25 23:02:44,577 - INFO - Start Training | Epoch: 5 | Dataset size: 940000 | Parameters: {'epochs': 25, 'early_stopping_patience': 5, 'validation_interval': 5, 'scheduler_step_by': 'epoch', 'images_to_visualize': 4, 'pos_enc': 5, 'accum_gradients': 4, 'batch_size': 4, 'beam_samples': 64, 'learning_rate': 5e-06, 'resize': 100} 
2025-03-25 23:20:57,606 - INFO - Epoch 0 - Training loss: 0.0098
2025-03-25 23:21:19,453 - INFO - Saved visualizations to logs/synthetic_2025-03-25_17-49-28/Sweep_4/samples/validation_epoch-0_*
2025-03-25 23:23:05,764 - INFO - Epoch 0 - Validation loss: 0.0085, MSE: 0.0085
2025-03-25 23:23:05,775 - INFO - Current model checkpoint saved to logs/synthetic_2025-03-25_17-49-28/Sweep_4/checkpoints/latest_model.pth
2025-03-25 23:23:05,786 - INFO - Best model checkpoint saved to logs/synthetic_2025-03-25_17-49-28/Sweep_4/checkpoints/best_model.pth (Validation Loss: 0.0085)
2025-03-25 23:23:05,787 - INFO - Avg time single epoch: 0h20m21s | Remaining time training:

Logger initialized. Logs will be saved to: logs/synthetic_2025-03-25_17-49-28/Sweep_5
INFO::Found 94 images within tilt series at ./data/synthetic/noisy-projections/*.tif.
INFO::Resized image resolution from (1000, 1000) to (100, 100). Note that strong downscaling can lead to loss of information.
INFO::Applied minmax normalization to tilt series. 
 Current max = 1.0000 | Current min = 0.0000
INFO:: tilt angles were loaded from ./data/synthetic/noisy-projections/tilt.rawtlt
INFO::Found 94 images within tilt series at ./data/synthetic/clean-projections/*.tif.
INFO::Resized image resolution from (1000, 1000) to (100, 100). Note that strong downscaling can lead to loss of information.
INFO::Applied minmax normalization to tilt series. 
 Current max = 1.0000 | Current min = 0.0000
INFO:: tilt angles were loaded from ./data/synthetic/clean-projections/tilt.rawtlt
INFO::Use 20% for validation, resulting in 18 projection images.
INFO::Found 94 images within tilt series at ./data/synthetic/clea

2025-03-26 00:45:20,080 - INFO - Start Training | Epoch: 5 | Dataset size: 940000 | Parameters: {'epochs': 25, 'early_stopping_patience': 5, 'validation_interval': 5, 'scheduler_step_by': 'epoch', 'images_to_visualize': 4, 'pos_enc': 5, 'accum_gradients': 4, 'batch_size': 8, 'beam_samples': 64, 'learning_rate': 5e-06, 'resize': 100} 
2025-03-26 00:54:54,720 - INFO - Epoch 0 - Training loss: 0.0118
2025-03-26 00:55:05,952 - INFO - Saved visualizations to logs/synthetic_2025-03-25_17-49-28/Sweep_5/samples/validation_epoch-0_*
2025-03-26 00:55:59,912 - INFO - Epoch 0 - Validation loss: 0.0092, MSE: 0.0092
2025-03-26 00:55:59,923 - INFO - Current model checkpoint saved to logs/synthetic_2025-03-25_17-49-28/Sweep_5/checkpoints/latest_model.pth
2025-03-26 00:55:59,934 - INFO - Best model checkpoint saved to logs/synthetic_2025-03-25_17-49-28/Sweep_5/checkpoints/best_model.pth (Validation Loss: 0.0092)
2025-03-26 00:55:59,935 - INFO - Avg time single epoch: 0h10m39s | Remaining time training:

Our automatic hyperparameter tuning is able to find the best performing set of hyperparameters based on the setting shown above. 

However, there can be scenarios, where additional flexibility is required. Therefore, you are able to change these hyperparameters in the following. 

**WARNING** This setting is for advanced users only. Please only change parameters here, if you know what you are doing. 

In [7]:
form = hyperparameter_tuner.edit_hyperparameters()
display(form)

In [8]:
best_config = hyperparameter_tuner.update_hyperparameters(form)
print_info(f"Will use following hyperparameters for future training: {best_config}")

[INFO]::Will use following hyperparameters for future training: {'learning_rate': 5e-05, 'batch_size': 8, 'resize': 100}


## 2.3. Training and Validation

Training in deep learning is the process where a model learns patterns from labeled data by optimizing its parameters through backpropagation. 
Validation involves using a separate dataset to evaluate the model's performance during training, ensuring it generalizes well to unseen data.
Hence, in this section we train and validate the model based on the provided data and hyperparameters resulting from the previous sweep.

If no sweep was conducted (not recommended for new datasets!), the default parameters, defined by the DL expert will be used. 

In case the training run was cancelled, it can be resumed from a previous checkpoint. To do so, you need to provide a model checkpoint in the text form below. You can find these checkpoints inside the runs logging directory (`<log-dir>/TrainingRun/checkpoints/latest_model.pth`). If you do not wish to resume training, you can ignore the text form below.

In [9]:
resume_widget = create_text_widget("Resume Training:","","If you wish to resume an earlier training, enter the path to the latest_model.pth file here.")
display(*resume_widget)

Text(value='', description='Resume Training:', style=TextStyle(description_width='initial'))

HTML(value='<b>Hint:</b> If you wish to resume an earlier training, enter the path to the latest_model.pth fil…

In [10]:


resume_training = resume_widget[0].value
if(resume_training):
    resume_training = Path(resume_training)
    if(resume_training.is_dir()):
        resume_training = Path(find_file(resume_training, "latest_model.pth")) 
        # resume_training = Path(os.path.join(resume_training,"latest_model.pth"))
    if(not resume_training.is_file()):
        logger.log_error(f"Could not find resume path at {resume_training}. Will start training from scatch.")
        resume_training = None
    else: 
        logger.log_info(f"Will resume training from {resume_training}")
else:
    resume_training = None
logger.init("TrainingRun")
model_trainer.resume_from_checkpoint = resume_training
model_trainer.prepare(best_config)


2025-03-26 01:38:54,487 - INFO - Hyperparameters saved to logs/synthetic_2025-03-25_17-49-28/TrainingRun/hyperparameters.json


Logger initialized. Logs will be saved to: logs/synthetic_2025-03-25_17-49-28/TrainingRun
INFO::Found 94 images within tilt series at ./data/synthetic/noisy-projections/*.tif.
INFO::Resized image resolution from (1000, 1000) to (100, 100). Note that strong downscaling can lead to loss of information.
INFO::Applied minmax normalization to tilt series. 
 Current max = 1.0000 | Current min = 0.0000
INFO:: tilt angles were loaded from ./data/synthetic/noisy-projections/tilt.rawtlt
INFO::Found 94 images within tilt series at ./data/synthetic/clean-projections/*.tif.
INFO::Resized image resolution from (1000, 1000) to (100, 100). Note that strong downscaling can lead to loss of information.
INFO::Applied minmax normalization to tilt series. 
 Current max = 1.0000 | Current min = 0.0000
INFO:: tilt angles were loaded from ./data/synthetic/clean-projections/tilt.rawtlt
INFO::Use 20% for validation, resulting in 18 projection images.
INFO::Found 94 images within tilt series at ./data/synthetic/

If you wish to train a model, execute the cell below. 

In [11]:
model_trainer.fit()

2025-03-26 01:38:57,082 - INFO - Start Training | Epoch: 25 | Dataset size: 940000 | Parameters: {'epochs': 25, 'early_stopping_patience': 5, 'validation_interval': 5, 'scheduler_step_by': 'epoch', 'images_to_visualize': 4, 'pos_enc': 5, 'accum_gradients': 4, 'batch_size': 8, 'beam_samples': 64, 'learning_rate': 5e-05, 'resize': 100} 
2025-03-26 01:48:22,734 - INFO - Epoch 0 - Training loss: 0.0070
2025-03-26 01:48:35,619 - INFO - Saved visualizations to logs/synthetic_2025-03-25_17-49-28/TrainingRun/samples/validation_epoch-0_*
2025-03-26 01:49:31,797 - INFO - Epoch 0 - Validation loss: 0.0068, MSE: 0.0068
2025-03-26 01:49:31,808 - INFO - Current model checkpoint saved to logs/synthetic_2025-03-25_17-49-28/TrainingRun/checkpoints/latest_model.pth
2025-03-26 01:49:31,820 - INFO - Best model checkpoint saved to logs/synthetic_2025-03-25_17-49-28/TrainingRun/checkpoints/best_model.pth (Validation Loss: 0.0068)
2025-03-26 01:49:31,821 - INFO - Avg time single epoch: 0h10m34s | Remaining t

np.float64(0.005827395319160439)

# 3. Model Evaluation
Evaluation in deep learning is the process of evaluating a trained model on a separate, unseen dataset to measure its final performance. It provides an unbiased assessment of the model's ability to generalize to new data.

## 3.1. Choose Model 

In this section we choose the model for testing. 
If you leave the `Model Path` empty in the text form below, it will use the last model trained.
Otherwise, you can define the path to the models best weights at `<log-path>/TrainingRun/checkpoints/best_model.pth` 

In [12]:
model_widget = create_text_widget("Model Path:","","If you wish to test a specific model, you can here define the path to its checkpoint. (For example: logs/tem-herpes_2025-02-03_11-42-43/TrainingRun/checkpoints)")
display(*model_widget)

Text(value='', description='Model Path:', style=TextStyle(description_width='initial'))

HTML(value='<b>Hint:</b> If you wish to test a specific model, you can here define the path to its checkpoint.…

## 3.2. Evaluate
We finally evaluate the provided model on the test set. We investigate following metrics: 

<span style="color:red"> Add a description of the used metrics here. </span>

<span style="color:green">

- **Accuracy (Acc)**: The percentage of correct predictions based on all predictions.

- **Mean Absolute Error (MAE)**: A distance metric to compute the difference between the prediction and the label. Measures the absolute difference between predicted and actual counts. The lower the metric, the better. 

- **Mean Absolute Error (MAE-class-name)**: A distance metric to compute the difference between the prediction and the label. Measures the absolute difference between predicted and actual counts for each class individually. This can help to find a specifically well/bad performing class. The lower the metric, the better.
</span>


<span style="color:red">Add a description of your visualizations here.</span>




These visualizations are saved to `<log-path>/Evaluate/samples/test_*`. 

If you wish to evaluate a model, execute the cell below.

In [14]:
from pathlib import Path 
start_evaluation = False
eval_model = model_widget[0].value
if(eval_model):
    eval_model = Path(eval_model)
    if(eval_model.is_dir()):
        eval_model = Path(find_file(eval_model, "best_model.pth")) 
    if(not eval_model.is_file()):
        logger.log_error(f"Could not find model at {eval_model}. Make sure to train a model before evaluation.")
        eval_model = None
    else: 
        start_evaluation = True
else:
    recent_logs = logger.get_most_recent_logs()
    eval_model = ""
    for dataname, log_path in recent_logs.items():
        if(dataname == Path(data_path).stem):
            eval_model = Path(log_path+"/TrainingRun/checkpoints/best_model.pth")
            if(not eval_model.is_file()):
                logger.log_error(f"Cound not find a trained model at {eval_model}. Make sure you fully train a model first before evaluating.")
            else:
                logger.log_info(f"Found most recent log at {eval_model}")
                start_evaluation = True
        else: 
            continue
    if(not start_evaluation):
        logger.log_error("Cound not find a trained model. Make sure you train a model first before evaluating.")
      
if(start_evaluation):
    model_trainer.load_checkpoint(eval_model)
    model_trainer.test()      


2025-03-26 07:50:23,226 - INFO - Found most recent log at logs/synthetic_2025-03-25_17-49-28/TrainingRun/checkpoints/best_model.pth


INFO::Found 94 images within tilt series at ./data/synthetic/noisy-projections/*.tif.
INFO::Resized image resolution from (1000, 1000) to (100, 100). Note that strong downscaling can lead to loss of information.
INFO::Applied minmax normalization to tilt series. 
 Current max = 1.0000 | Current min = 0.0000
INFO:: tilt angles were loaded from ./data/synthetic/noisy-projections/tilt.rawtlt
INFO::Found 94 images within tilt series at ./data/synthetic/clean-projections/*.tif.
INFO::Resized image resolution from (1000, 1000) to (100, 100). Note that strong downscaling can lead to loss of information.
INFO::Applied minmax normalization to tilt series. 
 Current max = 1.0000 | Current min = 0.0000
INFO:: tilt angles were loaded from ./data/synthetic/clean-projections/tilt.rawtlt
INFO::Use 20% for validation, resulting in 18 projection images.
INFO::Found 94 images within tilt series at ./data/synthetic/clean-projections/*.tif.
INFO::Resized image resolution from (1000, 1000) to (100, 100). N

2025-03-26 07:50:25,974 - INFO - Resumed training from checkpoint: logs/synthetic_2025-03-25_17-49-28/TrainingRun/checkpoints/best_model.pth (Validation Loss: 0.0058)


Logger initialized. Logs will be saved to: logs/synthetic_2025-03-25_17-49-28/Evaluate


2025-03-26 07:50:39,444 - INFO - Saved visualizations to logs/synthetic_2025-03-25_17-49-28/Evaluate/samples/test_*
2025-03-26 07:55:31,444 - INFO - Test loss: 0.0062
2025-03-26 07:55:32,172 - INFO - MSE: 0.0062


Max loaded data: 255
INFO::Original shape of volume (1000, 1000, 1000) was resized to (100, 100, 100)


Generate Tomogram: 100%|██████████| 68750/68750 [00:43<00:00, 1566.00it/s]
2025-03-26 07:56:16,821 - INFO - Test loss phantom: 0.0481
2025-03-26 07:56:16,833 - INFO - Evaluation Tomogram was saved to logs/synthetic_2025-03-25_17-49-28/Evaluate/samples/tomogram.tif
